In [1]:
from src import SQL_Executor, SQL_Queries

executor = SQL_Executor()
query = SQL_Queries.Vintage()

In [2]:
executor.execute_query(query.group_by_first_transaction())

,month,user_id
0,2023-12,10
1,2023-12,44
2,2023-12,63
3,2023-12,76
4,2023-12,206
...,...,...
12517,2024-11,22191
12518,2024-11,22192
12519,2024-11,22193
12520,2024-11,22195
